# Retrieval Augumented Generation (RAG) inference

***This notebook works best with the `conda_python3` on the `ml.t3.large` instance***.

---

At this point our slide deck data is ingested into Amazon OpenSearch Service Serverless collection. We are now ready to talk to our slide deck using a large multimodal model. We are using the [Anthropic’s Claude 3 Sonnet foundation model](https://aws.amazon.com/about-aws/whats-new/2024/03/anthropics-claude-3-sonnet-model-amazon-bedrock/) for this purpose.

## Step 1. Setup

Install the required Python packages and import the relevant files.

In [ ]:
import sys
!{sys.executable} -m pip install -r requirements.txt

In [1]:
import os
import io
import sys
import json
import glob
import boto3
import codecs
import logging
import requests
import botocore
import jsonlines
import numpy as np
import pandas as pd
import globals as g
from pathlib import Path
from typing import List, Dict
from IPython.display import Image
from urllib.parse import urlparse
from botocore.auth import SigV4Auth
from pandas.core.series import Series
from sagemaker import get_execution_role
from botocore.awsrequest import AWSRequest
from opensearchpy import OpenSearch, RequestsHttpConnection, AWSV4SignerAuth
from sagemaker.huggingface.model import HuggingFaceModel, HuggingFacePredictor
from utils import get_img_desc, download_image_from_url, encode_image_to_base64
from utils import get_cfn_outputs, get_text_embedding, find_similar_data

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [2]:
logging.basicConfig(format='[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)

## Step 2. Create an OpenSearch client and SageMaker Predictor object

We create an OpenSearch client so that we can query the vector database for embeddings (slides) similar to the questions that we might want to ask of our slide deck and then we run inference using the Claude 3 Sonnet model given the slide we retrieved from OpenSearch.

Get the name of the OpenSearch Service Serverless collection endpoint and index name from the CloudFormation stack outputs.

In [3]:
outputs = get_cfn_outputs(g.CFN_STACK_NAME)
host = outputs['MultimodalCollectionEndpoint'].split('//')[1]
index_name = outputs['OpenSearchIndexName']
logger.info(f"opensearchhost={host}, index={index_name}")

[2024-07-17 19:22:08,885] p9862 {725157277.py:5} INFO - opensearchhost=7uiiz7d87b3q8u2kfmtd.us-east-1.aoss.amazonaws.com, index=blog3slides-app1


In [4]:
session = boto3.Session()
credentials = session.get_credentials()
auth = AWSV4SignerAuth(credentials, g.AWS_REGION, g.OS_SERVICE)

os_client = OpenSearch(
    hosts = [{'host': host, 'port': 443}],
    http_auth = auth,
    use_ssl = True,
    verify_certs = True,
    connection_class = RequestsHttpConnection,
    pool_maxsize = 20
)

[2024-07-17 19:22:10,134] p9862 {credentials.py:1075} INFO - Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole


## Step 3. Read for RAG

We now have all the pieces for RAG. Here is how we _talk to our slide deck_.

1. Convert the user question into embeddings using the Titan Multimodal Embeddings model.

1. Find the most similar slide (image) corresponding to the the embeddings (for the user question) from the vector database (OpenSearch Serverless).

1. Now ask Claude 3 (via the Bedrock Endpoint) to answer the user question using the retrieved image for the most similar slide.


In [5]:
bedrock = boto3.client(service_name="bedrock-runtime", endpoint_url=g.TITAN_URL)

Use the following prompt template to make sure that the model only answers from the slides (images).

### Ask questions
Loop through questions in the jsonl file to -
1. Embed the question
2. Do a similarity search to retrive the closest image url and image description
3. Get final response from Claude by passing question and image description
4. Append responses to a list

Save the responses

In [6]:
llm_prompt: str = Path('prompts/image_response_prompt.txt').read_text()
print(llm_prompt)

In [7]:
responses_list = []
with jsonlines.open('qa.jsonl') as f:
    for line in f.iter():
        question: str = line['question']
        question_deckname: str = line['deck_name']
        question_deckurl: str = line['deck_url']
        text_embedding = get_text_embedding(bedrock, question, g.FMC_MODEL_ID)
        
        k = 4
        vector_db_response: Dict = find_similar_data(os_client, text_embedding, k, index_name, question_deckname, question_deckurl)
        response_count = len(vector_db_response.get('hits', {}).get('hits'))
        if response_count >= 1:
            for i in range(k):
                resp_deck_name = vector_db_response.get('hits', {}).get('hits')[i].get('_source').get('metadata').get('deck_name')
                resp_deck_url = vector_db_response.get('hits', {}).get('hits')[i].get('_source').get('metadata').get('deck_url')
                resp_img_url = vector_db_response.get('hits', {}).get('hits')[i].get('_source').get('image_url')

                logger.info(f"answering the question=\"{question}\" using the image \"{resp_img_url}\"")
                prompt = llm_prompt.format(question=question)
                img_path = download_image_from_url(resp_img_url, g.IMAGE_DIR)
                if img_path != "":
                    b64_img_path = encode_image_to_base64(img_path)
                    resp_text = get_img_desc(bedrock, b64_img_path, prompt)
                    
                    if resp_text.lower() != 'no answer':
                        response = {
                            "question": question,
                            "question_deckname": question_deckname,
                            "question_deckurl": question_deckurl,
                            "response": {
                                "resp_txt": resp_text,
                                "resp_img_url": resp_img_url,
                                "resp_deck_name": resp_deck_name,
                                "resp_deck_url": resp_deck_url
                            }
                        }
                        responses_list.append(response)
                        logger.info(f"appended response corresponding to {question}")
                        break

[2024-07-17 19:22:21,676] p9862 {base.py:259} INFO - POST https://7uiiz7d87b3q8u2kfmtd.us-east-1.aoss.amazonaws.com:443/blog3slides-app1/_search [status:200 request:0.136s]
[2024-07-17 19:22:21,679] p9862 {utils.py:82} INFO - received response from OpenSearch
[2024-07-17 19:22:21,681] p9862 {634680070.py:22} INFO - answering the question="In which year did Nestlé achieve higher Organic Growth, 2003 or 2004?" using the image "https://image.slidesharecdn.com/2012-02-20fy11roadshow-120221022442-phpapp02/95/feb-20-2012-nestl-2011-fullyear-roadshow-presentation-7-1024.jpg"
[2024-07-17 19:22:21,684] p9862 {utils.py:44} INFO - downloading image at https://image.slidesharecdn.com/2012-02-20fy11roadshow-120221022442-phpapp02/95/feb-20-2012-nestl-2011-fullyear-roadshow-presentation-7-1024.jpg
[2024-07-17 19:22:21,733] p9862 {utils.py:48} INFO - https://image.slidesharecdn.com/2012-02-20fy11roadshow-120221022442-phpapp02/95/feb-20-2012-nestl-2011-fullyear-roadshow-presentation-7-1024.jpg download

For each question, save the question and response

In [9]:
fpath: str = "responses-appr1.json"
json.dump(responses_list, codecs.open(fpath, 'w', encoding='utf-8'), 
          separators=(',', ':'), 
          sort_keys=True, 
          indent=4)
logger.info(f"saved responses for all questions in {fpath}")

[2024-07-17 19:47:32,000] p9862 {928541742.py:6} INFO - saved responses for all questions in responses-appr1.json
